<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Topic Modelling using Vantage and OpenAI
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233c'><b>Introduction:</b></p>

<p style='font-size:16px;font-family:Arial;color:#00233C'>In this comprehensive user demo, we will delve into the world of topic modeling using <b>Teradata Vantage</b> and <b>OpenAI LLM model</b>. This cutting-edge technology empowers businesses to uncover hidden insights from vast amounts of consumer complaints data, enabling them to identify trends, improve customer satisfaction, and enhance their overall brand reputation.</p> 

<p style='font-size:16px;font-family:Arial;color:#00233C'><b>Key Features:</b></p> 

<ol style='font-size:16px;font-family:Arial;color:#00233C'> 
    <li><b>Scalable Data Ingestion</b>: Seamlessly integrate and process large volumes of consumer complaints data from various sources, including OpenAI, into Teradata Vantage.</li> 
    <li><b>Advanced Topic Modelling</b>: Utilize state-of-the-art topic modeling algorithms to identify and categorize underlying themes and sentiments within the complaints data, providing actionable insights.</li> 
    <li><b>Real-time Analytics</b>: Leverage Teradata Vantage's real-time analytics capabilities to monitor and respond to emerging trends and issues in consumer complaints.</li> 
    <li><b>Customizable Dashboards</b>: Create tailored dashboards to visualize and track key performance indicators (KPIs) and metrics specific to your business needs.</li> 
    <li><b>Integration with OpenAI</b>: Seamlessly integrate with OpenAI to collect and analyze consumer complaints data from these platforms.</li> </ol> 

<p style='font-size:16px;font-family:Arial;color:#00233C'><b>Benefits:</b></p> 

<ol style='font-size:16px;font-family:Arial;color:#00233C'>
<li><b>Enhanced Customer Insights</b>: Gain a deeper understanding of customer concerns and preferences, enabling data-driven decision-making.</li> 
<li><b>Improved Customer Satisfaction</b>: Identify and address recurring issues, leading to increased customer satisfaction and loyalty.</li> 
<li><b>Competitive Advantage</b>: Stay ahead of the competition by proactively addressing consumer complaints and improving brand reputation.</li> 
<li><b>Cost Savings</b>: Reduce the financial burden of handling and resolving consumer complaints by identifying and addressing root causes.</li> 
<li><b>Data-Driven Decision-Making</b>: Make informed business decisions based on actionable insights derived from topic modeling and real-time analytics.</li> </ol>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Configuring OpenAI Model</li>
    <li>Exploring the data</li>
    <li>Topic Modelling</li>
    <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;background-color:#00233C;'>
<b style = 'font-size:20px;font-family:Arial;color:#00233c'>1. Configuring the environment</b>
<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.1 Downloading and installing additional software needed</b>

In [ ]:
%%capture
!pip install -r requirements.txt --upgrade --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Import the required libraries</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Data manipulation and analysis
import numpy as np
import pandas as pd

# Visualization
import plotly.express as px

# Progress bar
from tqdm import tqdm

# Machine learning and other utilities from Teradata
from teradataml import *

# OpenAI API
import openai
from openai import OpenAI

# Display settings
display.max_rows = 5
pd.set_option('display.max_colwidth', None)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
execute_sql('''SET query_band='DEMO=Topic_Modelling.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_cloud');"        # Takes 1 minute
%run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_local');"        # Takes 2 minutes

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Configuring OpenAI Model</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following cell will prompt us for the following information:</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li><b>OpenAI API Key</b>: Enter your OpenAI API Key</li>
</ul>

<a href="..//Openai_setup_api_key/Openai_setup_api_key.md" style="text-decoration:none;" target="_blank"><button style="font-size:16px;font-family:Arial;color:#fff;background-color:#00233C;border:none;border-radius:5px;cursor:pointer;height:50px;line-height:50px;display:flex;align-items:center;">OpenAI API Key Guide <span style="margin-left:10px;">&#8658;</span></button>
</a>

In [ ]:
# Prompt user for OpenAI API key securely
openai.api_key = getpass.getpass(prompt="\nPlease enter your OpenAI API key: ")

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>3.1 Initialize the OpenAI Model</b>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li>The code below initializes a OpenAI client.</li>
</ul>

In [ ]:
client = OpenAI(
    api_key = openai.api_key
)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Exploring the data</b>

In [ ]:
df = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Consumer_Complaints'))

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here we subset the data to get only the complaints related to Mortgage. We further analyze the issues of those complaints and pick the top 5 topics.</p>

In [ ]:
df = df[df.product == 'Mortgage']

In [ ]:
df.select(['issue', 'sub_issue', 'complaint_id']).groupby(['issue', 'sub_issue']).agg(['count']).sort('count_complaint_id', ascending = False)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>According to the result above, we can classify the issues into the following topics:</p>

<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Mortgage Application</b>: Applying or refinancing</li>
    <li><b>Payment Trouble</b>: Issues during payment</li>
    <li><b>Mortgage Closing</b>: Finalizing the mortgage</li>
    <li><b>Report Inaccuracy</b>: Incorrect information</li>
    <li><b>Payment Struggle</b>: Difficulty paying</li>
<ul>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Topic Modelling</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Topic modeling using Large Language Models (LLMs) revolutionizes the way we understand and categorize vast collections of text data. LLMs excel in understanding the semantics and context of words, enabling sophisticated topic modeling techniques.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Traditionally, topic modeling algorithms like Latent Dirichlet Allocation (LDA) rely on statistical patterns within documents to identify topics. However, LLMs offer a more nuanced approach. By leveraging their deep understanding of language, LLMs can extract complex themes and topics from unstructured text data with higher accuracy and flexibility.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>LLMs can generate coherent topics without needing predefined categories, making them ideal for exploratory analysis of diverse datasets. Moreover, their ability to capture subtle nuances in language allows for more precise topic identification, even in noisy or ambiguous texts.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Reasoning with a Chain of Thought</b>: Imagine you're trying to solve a problem. With a large language model, you start with an initial idea or question. Then, you use the model's capabilities to explore related concepts, gradually connecting them together. Each step builds upon the previous one, leading you closer to understanding or solving the problem. It's like putting together puzzle pieces, one by one, until you see the whole picture.</p>

In [ ]:
pd_df = df.to_pandas()

In [ ]:
pd_df['Predicted_Topic'] = ''
pd_df['Reasoning with Chain of Thought'] = ''

In [ ]:
for i in tqdm(range(len(pd_df))):
    prompt = f'''
    The following is text from a complaint:
 
    “{pd_df['consumer_complaint_narrative'][i]}”
 
    Give me reasoning as well as topic for this complaint
    Instructions for Reasoning:
    - Give me Reasoning in detail
    - Only one sentence reasoning
    Instructions for Topic:
    - The complaint falls into only one of the following Topics: Mortgage Application, Payment Trouble, Mortgage Closing, Report Inaccuracy, Payment Struggle
    - Only select one Topic

    My output comes in the format:
    Topic:
    Reasoning:
'''
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an assistant that detects topic a text and gives reasoning for the categorization as well."},
            {"role": "user", "content": prompt}
        ]
    )
    output = completion.choices[0].message.content
    topic = re.search('Topic:(.*)', output).group(1)
    reasoning = re.search('Reasoning:(.*)', output).group(1)
    pd_df['Predicted_Topic'][i] = topic
    pd_df['Reasoning with Chain of Thought'][i] = reasoning

In [ ]:
pd_df[['complaint_id', 'consumer_complaint_narrative', 'Predicted_Topic', 'Reasoning with Chain of Thought']].head(5)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Save the results back to Vantage.</p>

In [ ]:
copy_to_sql(df = pd_df, table_name = 'topic_prediction', if_exists = 'replace')

<hr style='height:1px;border:none;background-color:#00233C;'>
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>5.1 Number of Complaints by Predicted Topic</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'>A graph illustrating the Number of Complaints by Predicted Topic reveals that the majority of complaints are centered around Mortgage Application, while the fewest are related to Mortgage Closing.</p>

In [ ]:
grp_gen = DataFrame('topic_prediction').select(['Predicted_Topic','complaint_id']).groupby(['Predicted_Topic']).agg(['count']).to_pandas()

grp_gen = grp_gen.sort_values('count_complaint_id', ascending = False)[:10]

fig = px.bar(grp_gen, x='Predicted_Topic', y='count_complaint_id',
             labels={'count_complaint_id': 'Number of Complaints', 'Predicted_Topic': 'Predicted_Topic'},
             title='Number of Complaints by Predicted Topic')

# Add hover information
fig.update_traces(hovertemplate='Issue: %{x}<br>Number of Complaints: %{y:,}')

fig.show()

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>6. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ComplaintAnalysis');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial; color: #00233C;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>